This tutorial illustrates a few of the most common utility functions available in the `utils` directory.

# Time Conversions

Time conversion utilities exist between the four modes of GPS Week and time of week in seconds, GPS milliseconds since start of GPS Epoch, Unix milliseconds since start of unix epoch, and Python datetime objects (timezone assumed to be UTC if none provided).

Functionality exists for all 12 combinations between the four time instances, but here we show just one example of looping through each instance time.

In [ ]:
from datetime import datetime, timezone
import gnss_lib_py.utils.time_conversions as tc

Methods to convert `datetime` to the other types of time instances.

In [ ]:
time_now = datetime.now(tz=timezone.utc)
time_now

# convert datetime to GPS week and GPS time of week
gps_week, gps_tow = tc.datetime_to_tow(time_now)
print(f"gps week: {gps_week} gps tow: {gps_tow}")

# convert datetime to GPS milliseconds
gps_millis = tc.datetime_to_gps_millis(time_now)
print(f"GPS milliseconds: {gps_millis}")

# convert datetime to UNIX milliseconds
unix_millis = tc.datetime_to_unix_millis(time_now)
print(f"UNIX milliseconds: {unix_millis}")

Methods to convert GPS week and GPS time of week to the other types of time instances.

In [ ]:
# convert GPS week and GPS time of week to datetime
datetime = tc.tow_to_datetime(gps_week, gps_tow)
print("datetime in UTC: ",datetime.strftime("%d %B, %Y %H:%M:%S"))

# convert GPS week and GPS time to GPS milliseconds
gps_millis = tc.tow_to_gps_millis(gps_week, gps_tow)
print(f"GPS milliseconds: {gps_millis}")

# convert GPS week and GPS time to UNIX milliseconds
unix_millis = tc.tow_to_unix_millis(gps_week, gps_tow)
print(f"UNIX milliseconds: {unix_millis}")

Methods to convert GPS milliseconds to other types of time instances.

In [ ]:
# convert GPS milliseconds to datetime
datetime = tc.gps_millis_to_datetime(gps_millis)
print("datetime in UTC: ",datetime.strftime("%d %B, %Y %H:%M:%S"))

# convert GPS milliseconds to GPS week and GPS time
gps_week, gps_tow = tc.gps_millis_to_tow(gps_millis)
print(f"gps week: {gps_week} gps tow: {gps_tow}")

# convert GPS milliseconds to UNIX milliseconds
unix_millis = tc.gps_to_unix_millis(gps_millis)
print(f"UNIX milliseconds: {unix_millis}")

Methods to convert UNIX milliseconds to other types of time instances.

In [ ]:
# convert UNIX milliseconds to datetime
datetime = tc.unix_millis_to_datetime(unix_millis)
print("datetime in UTC: ",datetime.strftime("%d %B, %Y %H:%M:%S"))

# convert UNIX milliseconds to GPS week and GPS time
gps_week, gps_tow = tc.unix_millis_to_tow(unix_millis)
print(f"gps week: {gps_week} gps tow: {gps_tow}")

# convert GPS milliseconds to UNIX milliseconds
gps_millis = tc.unix_to_gps_millis(unix_millis)
print(f"GPS milliseconds: {gps_millis}")

# Coordinates

Find elevation and azimuth angle from receiver and satellite ECEF positions.

In [ ]:
from gnss_lib_py.utils.coordinates import ecef_to_el_az
from gnss_lib_py.parsers.android import AndroidDerived2022

# load Android Google Challenge data
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android_2022/device_gnss.csv --quiet -O "device_gnss.csv"
navdata = AndroidDerived2022("device_gnss.csv")
navdata_subset = navdata.where("gps_millis",navdata["gps_millis",0]) # only use data from first timestep

To calculate the elevation and azimuth, pass in the receiver and satellites' ECEF positions.

In [ ]:
pos_sv_m = navdata_subset[["x_sv_m","y_sv_m","z_sv_m"]].T
pos_rx_m = navdata_subset[["WlsPositionXEcefMeters",
                           "WlsPositionYEcefMeters",
                           "WlsPositionZEcefMeters"],0].reshape(1,-1)

calculated_el_az = ecef_to_el_az(pos_rx_m,pos_sv_m)
truth_el_az = navdata_subset[["el_sv_deg","az_sv_deg"]].T

We can now compare the calculated elevation and azimuth with their respective "truth" values included in the Google Decimeter Challenge 2022 dataset.

In [ ]:
for sat_idx in range(3):
    print(f"SV ID: {int(navdata_subset['sv_id',sat_idx])}")
    print(f"Calculated elevation: {calculated_el_az[sat_idx,0]}, Truth elevation: {truth_el_az[sat_idx,0]}")
    print(f"Calculated azimuth: {calculated_el_az[sat_idx,1]}, Truth azimuth: {truth_el_az[sat_idx,1]}")